# 네이버 부동산 긁어오기(crawling)
네이버 부동산 사이트에서 원하는 단지/평형의 매매/전세 최저가를 가져와서 정리  
모바일 버전이 더 간단해서 긁어오기 쉽기 때문에 모바일 버전에서 긁어옴
## 절차
- 사전에 작성한 긁어올 아파트 리스트(APTlist.cvs)를 불러온다(네이버 부동산의 아파트, 평형 코드를 조사해서 정리해둠)
- 각각의 매매/전세 최저가를 긁어온다
- 갭가격, 전세율, 평당가를 계산한다
- 엑셀(날짜.csv) 파일로 출력한다

## 방법 설명
네이버 부동산 모바일 버전에서 원하는 아파트와 평형을 검색하고, 낮은가격순으로 정렬하면 이러한 url로 들어가짐  
https://m.land.naver.com/complex/info/483?tradTpCd=B1&ptpNo=2&bildNo=&articleListYN=Y  
여기서 개발자 도구를 이용해서, 네트워크의 Fetch/XHR 항목에 보면 이러한 파일이 로딩됨  
https://m.land.naver.com/complex/getComplexArticleList?hscpNo=483&cortarNo=1168010300&tradTpCd=B1&ptpNo=2&order=point_&showR0=N&page=1  
이것이 실질적인 정보가 들어있는 파일인데, 아파트 번호(hscpNo)와 평형 번호(ptpNo)를 알면 url을 합성해서 직접 접근할 수 있음  
(여기서 아파트 번호는 483, 평형 번호는 2)
원하는 아파트들의 아파트 번호와 평형 번호를 직접 수집한 후, 최저가(맨 위 항목의 가격 - prcInfo 값)를 자동으로 긁어오는 것임  

## ↓ 고고 ↓ 

### 1. 필요한 라이브러리 불러오기

In [1]:
import requests                # 네이버 부동산에 자동 접속하기 위한 라이브러리
import json                    # 소스를 분석하기 위한 라이브러리 1
from bs4 import BeautifulSoup # 소스를 분석하기 위한 라이브러리 2
import pandas as pd           # 표 형식의 데이터를 다루기 위한 라이브러리
import re                     # 문자열을 편집하기 위한 라이브러리
import datetime               # 오늘 날짜를 구하기 위한 라이브러리
import time                   # 일시정지용 라이브러리
import random                 # 일시정지 시간 랜덤화를 위한 난수발생 라이브러리
import urllib3                # 경고메시지 안뜨게 하는 라이브러리
urllib3.disable_warnings()
pd.set_option('mode.chained_assignment',  None)

### 2. 아파트 리스트를 불러온 후, 출력할 표 양식을 만듬

In [2]:
apt = pd.read_csv("APTlist.csv") # 아파트 리스트 불러오기
apt = apt.fillna("")             # 동 미지정으로 빈칸인 경우 NaN 값이 들어있으므로 빈칸으로 바꿈
# apt
output = pd.DataFrame()          # 출력할 표 양식 선언(지금은 빈 표)
date = datetime.datetime.now()   # 오늘 날짜를 date 라는 변수(객체)에 저장함
apt['Date'] = date.date()        # 한 열씩 양식을 만들어나감
output['날짜'] = apt['Date'].copy()
output['아파트명'] = apt['aptName'].copy()
output['구'] = apt['Gu'].copy()
output['동'] = apt['Dong'].copy()
output['평형'] = apt['Type'].copy()
output['매매가(만원)'] = 0
output['전세가(만원)'] = 0
output['갭가격(만원)'] = 0
output['전세율'] = 0.0
output['평당가(만원)'] = 0

output

,날짜,아파트명,구,동,평형,매매가(만원),전세가(만원),갭가격(만원),전세율,평당가(만원)
0,2024-01-04,개포래미안포레스트,강남구,개포동,22,0,0,0,0.0,0
1,2024-01-04,개포래미안포레스트,강남구,개포동,26,0,0,0,0.0,0
2,2024-01-04,개포우성6차,강남구,개포동,18,0,0,0,0.0,0
3,2024-01-04,개포우성6차,강남구,개포동,22,0,0,0,0.0,0
4,2024-01-04,개포우성6차,강남구,개포동,26,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
197,2024-01-04,수성롯데캐슬더퍼스트,대구시,수성구,33,0,0,0,0.0,0
198,2024-01-04,현대힐스테이트,수원시,영통구,33,0,0,0,0.0,0
199,2024-01-04,광교중흥S클래스,수원시,영통구,35,0,0,0,0.0,0
200,2024-01-04,수원아이파크시티2단지,수원시,영통구,34,0,0,0,0.0,0


### 3.1. 각 아파트의 최저 매매가를 출력용 표에 기록
참고1: https://leesunkyu94.github.io/data%20%EB%A7%8C%EB%93%A4%EA%B8%B0/naver-real-estate/#  
참고2: https://m.blog.naver.com/inasie/221353956889

In [3]:
url = 'https://m.land.naver.com/complex/getComplexArticleList' # 네이버 부동산 모바일 url

for i in range(len(apt)):   # apt 의 각 행에 대해서 아래 내용 반복
    param = {                  # 아파트별로 세부 url 구성요소가 다르므로 아파트 리스트의 값을 이용해 만들어준다
        "hscpNo": apt.aptCode[i],  # 아파트명(코드)
        "tradTpCd": "A1",      # 매매
        "ptpNo": apt.typeCode[i],  # 평형 타입(코드)
        "bildNo" : apt.dongCode[i], # 동 지정(코드)
        "order": "prc",        # 낮은가격순 정렬
        "showR0": "N",
        "page": "1"
    }
    header = {                 # 크롤링 방지를 위해 네이버에서 보안 코드 같은걸 넣어놓았으므로 이를 넣어준다
        "Accept-Encoding": "gzip, deflate, br",
        "Host": "m.land.naver.com",
        "Referer": "https://m.land.naver.com/",
        "sec-ch-ua": "\"Chromium\";v=\"104\", \" Not A;Brand\";v=\"99\", \"Microsoft Edge\";v=\"104\"",
        "sec-ch-ua-moblie": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70"
    }
    resp = requests.get(url, params=param, headers=header, verify=False) # 페이지 소스를 받아온다

    resp.encoding = "utf-8-sig"    # 인코딩 utf-8로 바꿔줘야 한글이 안깨짐
    temp = json.loads(resp.text)   # json 형식의 소스를 해석(파싱)하여 temp 변수에 저장
    if len(temp["result"]["list"]) > 0:
        temp2 = temp["result"]["list"][0]["prcInfo"] # 목록 중 맨 윗줄(최저가 매물)의 가격(prcInfo)을 temp2 변수에 저장
    else:   # 매물이 없을 경우는 0원 처리
        temp2 = "0"
        
    token = temp2.split()  # 가격이 "8억 5,000" 식으로 써있으므로 억단위/만원단위를 분리해서 숫자로 계산해줌
    if len(token) > 1:
        price1 = int(re.sub("억", "", token[0]))
        price2 = int(re.sub(",", "", token[1]))
        price = price1 * 10000 + price2
    else:   # 가격이 "9억"인 경우 9 -> 90000으로 바꿔주는 작업
        price = int(re.sub("억", "", token[0])) * 10000
    
    output['매매가(만원)'][i] = price  # 출력용 표의 매매가 열에 기록
    
    print("({0}/{1}) {2} 아파트 {3}평 매매 최저가 가져오기 성공".format(i+1, len(apt), apt.aptName[i], apt.Type[i]))
    
    sleepT = 3 + random.random() * 3  
    time.sleep(sleepT)  # 차단 방지를 위해 3~6초 멈춤
    
output

(1/202) 개포래미안포레스트 아파트 22평 매매 최저가 가져오기 성공
(2/202) 개포래미안포레스트 아파트 26평 매매 최저가 가져오기 성공
(3/202) 개포우성6차 아파트 18평 매매 최저가 가져오기 성공
(4/202) 개포우성6차 아파트 22평 매매 최저가 가져오기 성공
(5/202) 개포우성6차 아파트 26평 매매 최저가 가져오기 성공
(6/202) 개포주공5단지 아파트 22평 매매 최저가 가져오기 성공
(7/202) 개포주공5단지 아파트 25평 매매 최저가 가져오기 성공
(8/202) 대청 아파트 17평 매매 최저가 가져오기 성공
(9/202) 대청 아파트 21평 매매 최저가 가져오기 성공
(10/202) 대청 아파트 24평 매매 최저가 가져오기 성공
(11/202) 디에이치아너힐즈 아파트 20평 매매 최저가 가져오기 성공
(12/202) 디에이치아너힐즈 아파트 25평 매매 최저가 가져오기 성공
(13/202) 디에이치퍼스티어아이파크 아파트 25평 매매 최저가 가져오기 성공
(14/202) 디에이치퍼스티어아이파크 아파트 33평 매매 최저가 가져오기 성공
(15/202) 래미안블레스티지 아파트 24평 매매 최저가 가져오기 성공
(16/202) 래미안블레스티지 아파트 34평 매매 최저가 가져오기 성공
(17/202) 성원대치2단지 아파트 14평 매매 최저가 가져오기 성공
(18/202) 성원대치2단지 아파트 17평 매매 최저가 가져오기 성공
(19/202) 성원대치2단지 아파트 20평 매매 최저가 가져오기 성공
(20/202) 개포자이프레지던스 아파트 24평 매매 최저가 가져오기 성공
(21/202) 개포자이프레지던스 아파트 33평 매매 최저가 가져오기 성공
(22/202) 개포우성1차 아파트 31평 매매 최저가 가져오기 성공
(23/202) 대치삼성1차 아파트 25평 매매 최저가 가져오기 성공
(24/202) 대치아이파크 아파트 23평 매매 최저가 가져오기 성공
(25/202) 대치현대 아파트 26평 매매 최저가 가져오기 성공
(26/202) 대치

,날짜,아파트명,구,동,평형,매매가(만원),전세가(만원),갭가격(만원),전세율,평당가(만원)
0,2024-01-04,개포래미안포레스트,강남구,개포동,22,167000,0,0,0.0,0
1,2024-01-04,개포래미안포레스트,강남구,개포동,26,186800,0,0,0.0,0
2,2024-01-04,개포우성6차,강남구,개포동,18,180000,0,0,0.0,0
3,2024-01-04,개포우성6차,강남구,개포동,22,180000,0,0,0.0,0
4,2024-01-04,개포우성6차,강남구,개포동,26,210000,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
197,2024-01-04,수성롯데캐슬더퍼스트,대구시,수성구,33,63000,0,0,0.0,0
198,2024-01-04,현대힐스테이트,수원시,영통구,33,51000,0,0,0.0,0
199,2024-01-04,광교중흥S클래스,수원시,영통구,35,138000,0,0,0.0,0
200,2024-01-04,수원아이파크시티2단지,수원시,영통구,34,62000,0,0,0.0,0


### 3.2. 각 아파트의 최저 전세가를 출력용 표에 기록

In [4]:
for i in range(len(apt)):   # apt 의 각 행에 대해서 아래 내용 반복
    param = {                  # 아파트별로 세부 url 구성요소가 다르므로 아파트 리스트의 값을 이용해 만들어준다
        "hscpNo": apt.aptCode[i],  # 아파트명(코드)
        "tradTpCd": "B1",      # 전세
        "ptpNo": apt.typeCode[i],  # 평형 타입(코드)
        "order": "prc",        # 낮은가격순 정렬
        "showR0": "N",
        "page": "1"
    }
    header = {                 # 크롤링 방지를 위해 네이버에서 보안 코드 같은걸 넣어놓았으므로 이를 넣어준다
        "Accept-Encoding": "gzip, deflate, br",
        "Host": "m.land.naver.com",
        "Referer": "https://m.land.naver.com/",
        "sec-ch-ua": "\"Chromium\";v=\"104\", \" Not A;Brand\";v=\"99\", \"Microsoft Edge\";v=\"104\"",
        "sec-ch-ua-moblie": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70"
    }
    resp = requests.get(url, params=param, headers=header, verify=False) # 페이지 소스를 받아온다

    resp.encoding = "utf-8-sig"    # 인코딩 utf-8로 바꿔줘야 한글이 안깨짐
    temp = json.loads(resp.text)   # json 형식의 소스를 해석(파싱)하여 temp 변수에 저장
    if len(temp["result"]["list"]) > 0:
        temp2 = temp["result"]["list"][0]["prcInfo"] # 목록 중 맨 윗줄(최저가 매물)의 가격(prcInfo)을 temp2 변수에 저장
    else:   # 매물이 없을 경우는 0원 처리
        temp2 = "0"
        
    token = temp2.split()  # 가격이 "8억 5,000" 식으로 써있으므로 억단위/만원단위를 분리해서 숫자로 계산해줌
    if len(token) > 1:
        price1 = int(re.sub("억", "", token[0]))
        price2 = int(re.sub(",", "", token[1]))
        price = price1 * 10000 + price2
    else:   # 가격이 "9억"인 경우 9 -> 90000으로 바꿔주는 작업
        price = int(re.sub("억", "", token[0])) * 10000
    
    output['전세가(만원)'][i] = price  # 출력용 표의 전세가 열에 기록
    
    print("({0}/{1}) {2} 아파트 {3}평 전세 최저가 가져오기 성공".format(i+1, len(apt), apt.aptName[i], apt.Type[i]))
    
    sleepT = 3 + random.random() * 3  
    time.sleep(sleepT)  # 차단 방지를 위해 3~5초 멈춤
    
output

(1/202) 개포래미안포레스트 아파트 22평 전세 최저가 가져오기 성공
(2/202) 개포래미안포레스트 아파트 26평 전세 최저가 가져오기 성공
(3/202) 개포우성6차 아파트 18평 전세 최저가 가져오기 성공
(4/202) 개포우성6차 아파트 22평 전세 최저가 가져오기 성공
(5/202) 개포우성6차 아파트 26평 전세 최저가 가져오기 성공
(6/202) 개포주공5단지 아파트 22평 전세 최저가 가져오기 성공
(7/202) 개포주공5단지 아파트 25평 전세 최저가 가져오기 성공
(8/202) 대청 아파트 17평 전세 최저가 가져오기 성공
(9/202) 대청 아파트 21평 전세 최저가 가져오기 성공
(10/202) 대청 아파트 24평 전세 최저가 가져오기 성공
(11/202) 디에이치아너힐즈 아파트 20평 전세 최저가 가져오기 성공
(12/202) 디에이치아너힐즈 아파트 25평 전세 최저가 가져오기 성공
(13/202) 디에이치퍼스티어아이파크 아파트 25평 전세 최저가 가져오기 성공
(14/202) 디에이치퍼스티어아이파크 아파트 33평 전세 최저가 가져오기 성공
(15/202) 래미안블레스티지 아파트 24평 전세 최저가 가져오기 성공
(16/202) 래미안블레스티지 아파트 34평 전세 최저가 가져오기 성공
(17/202) 성원대치2단지 아파트 14평 전세 최저가 가져오기 성공
(18/202) 성원대치2단지 아파트 17평 전세 최저가 가져오기 성공
(19/202) 성원대치2단지 아파트 20평 전세 최저가 가져오기 성공
(20/202) 개포자이프레지던스 아파트 24평 전세 최저가 가져오기 성공
(21/202) 개포자이프레지던스 아파트 33평 전세 최저가 가져오기 성공
(22/202) 개포우성1차 아파트 31평 전세 최저가 가져오기 성공
(23/202) 대치삼성1차 아파트 25평 전세 최저가 가져오기 성공
(24/202) 대치아이파크 아파트 23평 전세 최저가 가져오기 성공
(25/202) 대치현대 아파트 26평 전세 최저가 가져오기 성공
(26/202) 대치

,날짜,아파트명,구,동,평형,매매가(만원),전세가(만원),갭가격(만원),전세율,평당가(만원)
0,2024-01-04,개포래미안포레스트,강남구,개포동,22,167000,55000,0,0.0,0
1,2024-01-04,개포래미안포레스트,강남구,개포동,26,186800,90000,0,0.0,0
2,2024-01-04,개포우성6차,강남구,개포동,18,180000,0,0,0.0,0
3,2024-01-04,개포우성6차,강남구,개포동,22,180000,0,0,0.0,0
4,2024-01-04,개포우성6차,강남구,개포동,26,210000,45000,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
197,2024-01-04,수성롯데캐슬더퍼스트,대구시,수성구,33,63000,35000,0,0.0,0
198,2024-01-04,현대힐스테이트,수원시,영통구,33,51000,33000,0,0.0,0
199,2024-01-04,광교중흥S클래스,수원시,영통구,35,138000,75000,0,0.0,0
200,2024-01-04,수원아이파크시티2단지,수원시,영통구,34,62000,40000,0,0.0,0


### 4. 갭가격, 전세율, 평당가 계산

In [5]:
for i in range(len(output)):  # 출력 표의 각 행에 대해서 아래 내용 반복
    
    if output['매매가(만원)'][i] > 0:  # 갭가격, 평당가는 매매 매물이 있을때만 계산
        output['갭가격(만원)'][i] = output['매매가(만원)'][i] - output['전세가(만원)'][i]
        output['평당가(만원)'][i] = output['매매가(만원)'][i] / output['평형'][i]
    else:
        output['갭가격(만원)'][i] = "매매매물없음"
        output['평당가(만원)'][i] = "매매매물없음"
    
    if output['매매가(만원)'][i] == 0:  # 전세율은 매매, 전세 매물이 다 있을때만 계산
        output['전세율'][i] = "매매매물없음"
    elif output['전세가(만원)'][i] == 0:
        output['전세율'][i] = "전세매물없음"        
    else:
        output['전세율'][i] = output['전세가(만원)'][i] / output['매매가(만원)'][i]  # % 형식으로 바꾸는건 구글 시트에서 직접 하기
        
output

,날짜,아파트명,구,동,평형,매매가(만원),전세가(만원),갭가격(만원),전세율,평당가(만원)
0,2024-01-04,개포래미안포레스트,강남구,개포동,22,167000,55000,112000,0.329341,7590
1,2024-01-04,개포래미안포레스트,강남구,개포동,26,186800,90000,96800,0.481799,7184
2,2024-01-04,개포우성6차,강남구,개포동,18,180000,0,180000,전세매물없음,10000
3,2024-01-04,개포우성6차,강남구,개포동,22,180000,0,180000,전세매물없음,8181
4,2024-01-04,개포우성6차,강남구,개포동,26,210000,45000,165000,0.214286,8076
...,...,...,...,...,...,...,...,...,...,...
197,2024-01-04,수성롯데캐슬더퍼스트,대구시,수성구,33,63000,35000,28000,0.555556,1909.09
198,2024-01-04,현대힐스테이트,수원시,영통구,33,51000,33000,18000,0.647059,1545.45
199,2024-01-04,광교중흥S클래스,수원시,영통구,35,138000,75000,63000,0.543478,3942.86
200,2024-01-04,수원아이파크시티2단지,수원시,영통구,34,62000,40000,22000,0.645161,1823.53


### 5. 오늘날짜 이름의 파일로 출력

In [6]:
output.to_csv(date.strftime('%Y%m%d') + ".csv", index=False, encoding="utf-8-sig")